# 問卷分析
- 卷 A, B: 前三題為正臉照, 後二題為側臉照 
- 卷 C, D: 前二題為正臉照, 後三題為側臉照

#### 目標
- 正臉照的結果
- 側臉照的結果
- 整體的結果

In [1]:
import pandas as pd
import numpy as np

In [2]:
pathA = 'survey/拯救痘痘臉-A (回覆).xlsx'
pathB = 'survey/拯救痘痘臉-B (回覆).xlsx'
pathC = 'survey/拯救痘痘臉-C (回覆).xlsx'
pathD = 'survey/拯救痘痘臉-D (回覆).xlsx'

In [3]:
A = pd.read_excel(pathA)
B = pd.read_excel(pathB)
C = pd.read_excel(pathC)
D = pd.read_excel(pathD)

# 各資料集的數量
print('A:', len(A))
print('B:', len(B))
print('C:', len(C))
print('D:', len(D))

A: 57
B: 43
C: 29
D: 27


In [4]:
# 將很差, 差, 一般, 好, 很好 轉換成 1, 2, 3, 4, 5
map = {
    '很差': 1,
    '差': 2,
    '普通': 3,
    '好': 4,
    '很好': 5
}
A = A.replace(map)
B = B.replace(map)
C = C.replace(map)
D = D.replace(map)
A.head(3)

,時間戳記,法1 [去除效果],法1 [自然度],法1 [真實度],法2 [去除效果],法2 [自然度],法2 [真實度],法3 [去除效果],法3 [自然度],法3 [真實度],...,法2 [真實度].4,法3 [去除效果].4,法3 [自然度].4,法3 [真實度].4,您對以上照片的整體觀感有何其他看法？.1,您對以上照片的整體觀感有何其他看法？.2,您對以上照片的整體觀感有何其他看法？.3,您對以上照片的整體觀感有何其他看法？.4,您的名字或暱稱 (選填),想說的話 (選填)
0,2024-06-13 07:20:02.697,4,3,3,4,4,4,3,4,4,...,4,3,4,4,他皮膚比較好 感覺三個方法效果都還行（？）,他幾乎沒痘痘吧😂,NaN,NaN,上緣,期末辛苦ㄌ
1,2024-06-13 09:40:37.257,4,4,4,5,4,4,3,4,5,...,4,3,5,5,NaN,NaN,NaN,NaN,你很吵,法一該不會是橡皮擦吧
2,2024-06-13 11:49:09.776,5,2,2,5,4,4,4,2,2,...,4,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 將欄位名稱為'法1[去除效果]'的資料取出
method = ['法1', '法2', '法3']
effect = [' [去除效果]', ' [自然度]', ' [真實度]']

group1 = ['', '.1', '.2'] # 問卷AB正臉
group2 = ['.3', '.4'] # 問卷AB側臉
group3 = ['', '.1'] # 問卷CD正臉
group4 = ['.2', '.3', '.4'] # 問卷CD側臉
group5 = ['', '.1', '.2', '.3', '.4'] #正臉側臉一起看

In [6]:
# 輸入單一問卷, 方法, 效果, 組別, 回傳該組的加總
def get_effect(data, m, e, g):
    columns = [data[m + e + i] for i in g]
    return np.hstack(columns)  
    

In [7]:
# 輸入單一問卷, 回傳不同方法
def get_all_effect(data, method, effect, group):
    results = {} 
    for m in method:
        for e in effect:
            key = m + e 
            data_array = get_effect(data, m, e, group)
            results[key] = data_array

    results['樣本數'] = len(data)
    results['題目數'] = len(group)
    
    return results

In [8]:
def sum_and_print_effects(*results):
    combined_results = {}
    for result in results:
        for key, data_array in result.items():
            if key not in ['樣本數', '題目數']:
                if key in combined_results:
                    combined_results[key] = np.concatenate((combined_results[key], data_array))
                else:
                    combined_results[key] = data_array
    
    for key, data_array in combined_results.items():
        average = np.mean(data_array)
        std_dev = np.std(data_array)
        print(f"{key}: 平均值 = {average:.2f}, 標準差 = {std_dev:.2f}")

In [9]:
# 正臉照
A_forward = get_all_effect(A, method, effect, group1)
B_forward = get_all_effect(B, method, effect, group1)
C_forward = get_all_effect(C, method, effect, group3)
D_forward = get_all_effect(D, method, effect, group3)

# 側臉照
A_side = get_all_effect(A, method, effect, group2)
B_side = get_all_effect(B, method, effect, group2)
C_side = get_all_effect(C, method, effect, group4)
D_side = get_all_effect(D, method, effect, group4)

# 正臉側臉一起看
A_all = get_all_effect(A, method, effect, group5)
B_all = get_all_effect(B, method, effect, group5)
C_all = get_all_effect(C, method, effect, group5)
D_all = get_all_effect(D, method, effect, group5)

In [10]:
# 正臉照
print("正臉照:")
sum_and_print_effects(A_forward, B_forward, C_forward, D_forward)


正臉照:
法1 [去除效果]: 平均值 = 3.55, 標準差 = 0.98
法1 [自然度]: 平均值 = 2.53, 標準差 = 1.13
法1 [真實度]: 平均值 = 2.58, 標準差 = 1.11
法2 [去除效果]: 平均值 = 4.10, 標準差 = 0.66
法2 [自然度]: 平均值 = 3.73, 標準差 = 0.90
法2 [真實度]: 平均值 = 3.63, 標準差 = 0.95
法3 [去除效果]: 平均值 = 2.83, 標準差 = 0.98
法3 [自然度]: 平均值 = 3.22, 標準差 = 0.96
法3 [真實度]: 平均值 = 3.21, 標準差 = 1.00


In [11]:
# 側臉照
print("側臉照:")
sum_and_print_effects(A_side, B_side, C_side, D_side)

側臉照:
法1 [去除效果]: 平均值 = 3.19, 標準差 = 1.07
法1 [自然度]: 平均值 = 2.16, 標準差 = 0.97
法1 [真實度]: 平均值 = 2.23, 標準差 = 1.00
法2 [去除效果]: 平均值 = 3.80, 標準差 = 0.87
法2 [自然度]: 平均值 = 3.73, 標準差 = 0.81
法2 [真實度]: 平均值 = 3.67, 標準差 = 0.87
法3 [去除效果]: 平均值 = 3.40, 標準差 = 0.90
法3 [自然度]: 平均值 = 3.27, 標準差 = 0.97
法3 [真實度]: 平均值 = 3.27, 標準差 = 0.98


In [12]:
# 正臉側臉一起看
sum_and_print_effects(A_all, B_all, C_all, D_all)

法1 [去除效果]: 平均值 = 3.38, 標準差 = 1.04
法1 [自然度]: 平均值 = 2.36, 標準差 = 1.07
法1 [真實度]: 平均值 = 2.42, 標準差 = 1.07
法2 [去除效果]: 平均值 = 3.96, 標準差 = 0.78
法2 [自然度]: 平均值 = 3.73, 標準差 = 0.86
法2 [真實度]: 平均值 = 3.65, 標準差 = 0.92
法3 [去除效果]: 平均值 = 3.10, 標準差 = 0.99
法3 [自然度]: 平均值 = 3.24, 標準差 = 0.97
法3 [真實度]: 平均值 = 3.24, 標準差 = 0.99
